## Постановка задачи

Красивое введение от организаторов:

> Правда ли, что если проанализировать данные по оплатам, бронированиям и отменам для крупной сети гостиниц и попытаться найти скрытые зависимости с датами заезда, формой оплаты и другими признаками, то можно создать новый банковский продукт, который поможет с организацией отдыха для студентов? <br> Посмотрим, смогут ли алгоритмы машинного обучения и искусственного интеллекта ответить на этот вопрос.

На практике:

Данные представляют собой информацию о бронировании номеров различных категорий в разных гостиницах. Гостиницы с номерами 1 и 2 - один регион РФ, с номерами 3 и 4 - другой регион РФ.
<br>
Задача: предсказать отмену бронирования (бинарная классификация).
<br>
Структура данных следующая:
<br>
* № брони - идентификатор брони <br>
* Номеров - количество номеров в бронировании <br>
* Стоимость - стоимость номеров в рублях <br>
* Внесена предоплата - сумма внесенной предоплаты <br>
* Способ оплаты - один из 12 способов оплаты <br>
* Дата бронирования - дата бронирования с точностью до минуты <br>
* Дата отмены - дата отмены бронирования с точностью до минуты, если было <br>
* Заезд - дата заезда с точностью до дня <br>
* Ночей - количество ночей <br>
* Выезд - дата выезда с точностью до дня <br>
* Источник - онлайн-канал продаж <br>
* Статус брони - один из 5 статусов <br>
* Категория номера - описание категории номера. Если бронировалось несколько номеров, то идет сплошное описание с нумерацией. <br>
* Гостей - число гостей <br>
* Гостиница - номер гостиницы <br>

Целевое поле - Дата отмены: если поле заполнено, то это
соответствует целевому значению 1, иначе - О. Статусы в поле Статус брони уточняют состояние и носят справочный характер (но могут помочь при построении модели).
<br>
Метрика качества - ROC AUC.
<br>

## Обработка данных + новые признаки

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
data = pd.read_csv("drive/MyDrive/train.csv", delimiter=',', index_col=0)
data.head(2)

,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-4-20 20:37:30,2023-4-20 20:39:15,2023-4-28 15:00:00,3,2023-5-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-6-18 14:17:02,NaN,2022-7-11 15:00:00,2,2022-7-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1


In [3]:
def data_engineering(dataset, type_="train", write_csv=False):

    data = dataset.copy()

    # уникальный для каждой записи - удаляем
    data.drop("№ брони", axis=1, inplace=True)

    # кодируем - LabelEncoding
    mapping_pay_way = dict()
    for i, key in enumerate(data['Способ оплаты'].unique()):
        mapping_pay_way[key] = i

    data['Способ оплаты'] = data['Способ оплаты'].map(mapping_pay_way)

    mapping_source = dict()
    for i, key in enumerate(data['Источник'].unique()):
        mapping_source[key] = i

    data['Источник'] = data['Источник'].map(mapping_source)

    mapping_category = dict()
    for i, key in enumerate(data['Категория номера'].unique()):
        mapping_category[key] = i

    data['Категория номера'] = data['Категория номера'].map(mapping_category)

    if type_ == "train":
        mapping_status = dict()
        for i, key in enumerate(data['Статус брони'].unique()):
            mapping_status[key] = i

    data['Статус брони'] = data['Статус брони'].map(mapping_status)

    # разбиваем даты
    data['Заезд'] = pd.to_datetime(data['Заезд'])
    data['Выезд'] = pd.to_datetime(data['Выезд'])

    if type_ == "train":
      data['Дата отмены'] = pd.to_datetime(data['Дата отмены'])

    data['Заезд-выходной'] = [int(i.weekday() in [5, 6]) for i in data['Заезд']]
    data['Выезд-выходной'] = [int(i.weekday() in [5, 6]) for i in data['Выезд']]

    data['Заезд_день'] = [int(str(i.date()).split('-')[2]) for i in data['Заезд']]
    data['Заезд_месяц'] = [int(str(i.date()).split('-')[1]) for i in data['Заезд']]
    data['Заезд_год'] = [int(str(i.date()).split('-')[0]) for i in data['Заезд']]

    data['Выезд_день'] = [int(str(i.date()).split('-')[2]) for i in data['Выезд']]
    data['Выезд_месяц'] = [int(str(i.date()).split('-')[1]) for i in data['Выезд']]
    data['Выезд_год'] = [int(str(i.date()).split('-')[0]) for i in data['Выезд']]

    if type_ == "train":
        data['Отмена'] = [int(str(i) != 'NaT') for i in data['Дата отмены']]

    # запись изменённых
    if write_csv:
        data.to_csv("new_train.csv")

    # убираем лишние данные
    X = data.copy()

    X = X.drop('Дата бронирования', axis=1)
    if type_ == "train":
        X = X.drop('Дата отмены', axis=1)
        X = X.drop('Статус брони', axis=1)

    X = X.drop('Заезд', axis=1)
    X = X.drop('Выезд', axis=1)

    # разделяем x и y
    if type_ == "train":
        Y = X["Отмена"].copy()
        X = X.drop('Отмена', axis=1)

        return X, Y

    return X

## Тест моделей

### Библиотеки и данные

In [4]:
!pip install catboost

In [24]:
import copy
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from catboost import CatBoostClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [6]:
X, Y = data_engineering(data, type_="train")

In [7]:
X.head(2)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Ночей,Источник,Категория номера,Гостей,Гостиница,Заезд-выходной,Выезд-выходной,Заезд_день,Заезд_месяц,Заезд_год,Выезд_день,Выезд_месяц,Выезд_год
0,1,25700.0,0,0,3,0,0,2,1,0,0,28,4,2023,1,5,2023
1,1,24800.0,12400,1,2,1,0,2,1,0,0,11,7,2022,13,7,2022


In [8]:
n = X.shape[1]    # количество признаков
n

17

In [9]:
X = X.values
Y = Y.values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)

### Модели CatBoost и SklearnMLP

In [ ]:
# CatBoost 0.7609396443581299
# model = CatBoostClassifier(iterations=2000,
#                            depth=2,
#                            learning_rate=1,
#                            loss_function='Logloss',
#                            verbose=False, task_type='CPU')
# model.fit(X_train, y_train)
# roc_auc_score(y_test, model.predict(X_test))

In [11]:
# MLP 0.7596100531551162
scores = []
for i in range(1, 5):
    for j in range(1, 5):
        for k in range(1, 5):
            model = MLPClassifier(hidden_layer_sizes=(i * 20, j * 20, k * 20))
            model.fit(X_train, y_train)
            scores.append([roc_auc_score(y_test, model.predict(X_test)), (i * 20, j * 20, k * 20)])

    best_score = max(scores, key=lambda x: x[0])
    print(f"Best: ({best_score[1][0], best_score[1][1], best_score[1][2]}) hidden size with ", best_score[0])

Best: ((20, 60, 40)) hidden size with  0.7596100531551162
Best: ((20, 60, 40)) hidden size with  0.7596100531551162
Best: ((20, 60, 40)) hidden size with  0.7596100531551162
Best: ((20, 60, 40)) hidden size with  0.7596100531551162


### Torch модель на кастомной функции потерь

AUC - доля упорядоченных пар $(x_i, x_j)$:
$$ AUC(w)= \frac{1}{l_-}\sum_{i=1}^lTPR_i = \frac{1}{l_-l_+}\sum_{i=1}^l\sum_{j=1}^l[y_i<y_j][g(x_i, w)<g(x_j, w)] \to max_w, $$ где $g(x,w)$ - предсказание модели на объекте $x$ с параметрами $w$.<br>
Функционал качества следующего вида: <br>
$$ Q(w) = \sum_{i, j: y_i < y_j} L(g(x_j, w) - g(x_i, w)) \to min_w,$$
где $L(M)$ - убывающая функция оступа.

In [12]:
def L(margin):
    # экспоненциальная функция отступа
    return torch.exp(-1 * margin)


def loss_auc(prediction_1, prediction_2, answer_1, answer_2):
    if answer_2 > answer_1:
        M = prediction_2 - prediction_1
    else:
        M = prediction_1 - prediction_2
    return L(M)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [14]:
class NeuralNetwork(torch.nn.Module):
    @property
    def device(self):
        for p in self.parameters():
            return p.device

    def __init__(self, input_dim,
                 num_layers=0, hidden_dim=32, output_dim=1, p=0.2):

        super().__init__()

        self.layers = torch.nn.Sequential()

        prev_size = input_dim
        for i in range(num_layers):
            self.layers.add_module('layer{}'.format(i),
                                  torch.nn.Linear(prev_size, hidden_dim))
            self.layers.add_module('relu{}'.format(i), torch.nn.ReLU())
            self.layers.add_module('dropout{}'.format(i), torch.nn.Dropout(p=p))
            prev_size = hidden_dim

        self.layers.add_module('classifier',
                               torch.nn.Linear(prev_size, output_dim))

    def forward(self, input):
        return self.layers(input)

In [15]:
class Dataset_(Dataset):
    def __init__(self, x, y, test=False):
        self.X = x.astype(np.float64)
        self.y = y
        self.test = test        # флаг на режим train / test

        self.X_true = None
        self.X_false = None

        if not test:
            self.X_true = self.X[y==1, :]                           # объекты положительного класса
            self.X_false = self.X[y==0, :]                          # объекты отрицательного класса
            self.size = len(self.X_true) * len(self.X_false)

    def __len__(self):
        if self.test:
            return len(self.X)
        return self.size

    def __getitem__(self, i):
        if self.test:
            return self.X[i, :], self.y[i]

        # на тесте составляем все возможные пары
        # (отрицательный класс, положительный класс)
        tr_ind = i % len(self.X_true)
        fls_ind = i % len(self.X_false)
        return self.X_false[fls_ind, :], self.X_true[tr_ind, :]

In [30]:
# стандартизация для устойчивости
X_train, X_test, y_train, y_test = train_test_split(
    StandardScaler().fit_transform(X), Y, test_size=0.2, random_state=42)

train_dataset = Dataset_(X_train, y_train, test=False)
test_dataset = Dataset_(X_test, y_test, test=True)

train_dataloader = DataLoader(train_dataset, batch_size=None, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=None)

In [31]:
def test(dataloader, model, loss_fn):
    model.eval()
    y_test, pred_test = [], []

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X.float())
            pred = int(pred >= 0)

            y_test += [y.tolist()]
            pred_test += [pred]

    res = roc_auc_score(y_test, pred_test)
    print(f"[Test] ROC-AUC: {(res)}\n")

    return res

In [32]:
def train(dataloader, model, loss_fn, optimizer, test_dataloader):
    best_model = None
    best_score = 0
    counts_without_increase = 0

    size = len(dataloader.dataset)
    model.train()
    for batch, (x1, x2) in enumerate(dataloader):
        x1, x2 = x1.to(device), x2.to(device)

        pred1 = model(x1.float())
        pred2 = model(x2.float())
        loss = loss_fn(pred1, pred2, 0, 1)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10000 == 0:
            loss, current = loss.item(), (batch + 1)
            print(f"[{current}/{size}]", end=": ")
            score = test(test_dataloader, model, loss_fn)

            if score > best_score:
                best_model = copy.deepcopy(model)
                best_score = score
            else:
                counts_without_increase += 1

        if counts_without_increase > 100:
            break

    return best_model, best_score

In [34]:
model = NeuralNetwork(n, num_layers=3, hidden_dim=50, p=0.2).to(device)
loss_fn = loss_auc

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=1e-3)

In [ ]:
# 0.765818259719188

In [35]:
best_model, best_score = train(train_dataloader, model, loss_fn, optimizer, test_dataloader)
best_score

[1/70316338]: [Test] ROC-AUC: 0.4675887178765344

[10001/70316338]: [Test] ROC-AUC: 0.5184046855250506

[20001/70316338]: [Test] ROC-AUC: 0.5200847447955255

[30001/70316338]: [Test] ROC-AUC: 0.5286851217983529

[40001/70316338]: [Test] ROC-AUC: 0.5834713172558683

[50001/70316338]: [Test] ROC-AUC: 0.5195828193610544

[60001/70316338]: [Test] ROC-AUC: 0.5966331288857017

[70001/70316338]: [Test] ROC-AUC: 0.5410558837263933

[80001/70316338]: [Test] ROC-AUC: 0.5705683146432882

[90001/70316338]: [Test] ROC-AUC: 0.7197727235981516

[100001/70316338]: [Test] ROC-AUC: 0.7486912133803668

[110001/70316338]: [Test] ROC-AUC: 0.7405336767952096

[120001/70316338]: [Test] ROC-AUC: 0.7333829631623346

[130001/70316338]: [Test] ROC-AUC: 0.7198419136983083

[140001/70316338]: [Test] ROC-AUC: 0.7341616489974332

[150001/70316338]: [Test] ROC-AUC: 0.7398937278104257

[160001/70316338]: [Test] ROC-AUC: 0.6558463019199182

[170001/70316338]: [Test] ROC-AUC: 0.7546615343938994

[180001/70316338]: [Test

0.76072648654098

In [38]:
best_model.eval()
res = best_model(torch.from_numpy(X_test.astype(np.float32)).to(device)).reshape(-1).tolist()
roc_auc_score(y_test, [int(i > 0) for i in res])

0.76072648654098